In [1]:
from urllib.request import urlopen

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import re

In [3]:
# Determine if entry is a useful table entry
def is_beer_entry(table_row):
    #find all cells that are within a table
    row_cells = table_row.find_all("td")
    #get the cells in the first column, which should be beer_id
    beer_id = get_beer_id(row_cells[0].text)
    #check if the entry looks like a beer entry
    return ( len(row_cells) == 8 and beer_id)


In [4]:
#Get valid beer_ids from the table
def get_beer_id(cell_value):
    #regex to find valid beer id
    r = re.match("^(\d{1,4})\.$", cell_value)
    if r and len(r.groups()) == 1:
        beer_id = r.group(1)
        return int(beer_id)
    else:
        return None
   

In [5]:
def get_all_beers(html_soup):
    beers=[]
    all_rows_in_html_page=html_soup.find_all("tr")
    for table_row in all_rows_in_html_page:
        if is_beer_entry(table_row):
            row_cells = table_row.find_all("td")
            beer_entry = {
                "id":get_beer_id(row_cells[0].text),
                "name":row_cells[1].text,
                "brewery_name":row_cells[2].text,
                "brewery_location":row_cells[3].text,
                "style":row_cells[4].text,
                "size":row_cells[5].text,
                "abv":row_cells[6].text,
                "ibu":row_cells[7].text
            }
            beers.append(beer_entry)
    return beers


In [6]:
html = urlopen("http://craftcans.com/db.php?search=all&sort=beerid&ord=desc&view=text")
html_soup = BeautifulSoup(html,'html.parser')
beers_list = get_all_beers(html_soup)

In [7]:
df = pd.DataFrame(beers_list)
df.head(5)

,abv,brewery_location,brewery_name,ibu,id,name,size,style
0,4.5%,"Minneapolis, MN",NorthGate Brewing,50,2692,Get Together,16 oz.,American IPA
1,4.9%,"Minneapolis, MN",NorthGate Brewing,26,2691,Maggie's Leap,16 oz.,Milk / Sweet Stout
2,4.8%,"Minneapolis, MN",NorthGate Brewing,19,2690,Wall's End,16 oz.,English Brown Ale
3,6.0%,"Minneapolis, MN",NorthGate Brewing,38,2689,Pumpion,16 oz.,Pumpkin Ale
4,6.0%,"Minneapolis, MN",NorthGate Brewing,25,2688,Stronghold,16 oz.,American Porter


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2410 entries, 0 to 2409
Data columns (total 8 columns):
abv                 2410 non-null object
brewery_location    2410 non-null object
brewery_name        2410 non-null object
ibu                 2410 non-null object
id                  2410 non-null int64
name                2410 non-null object
size                2410 non-null object
style               2410 non-null object
dtypes: int64(1), object(7)
memory usage: 150.7+ KB


In [69]:
breweries = df[["brewery_location","brewery_name"]]
breweries = breweries.drop_duplicates().reset_index(drop=True)
breweries["id"] = breweries.index
breweries.head(5)

,brewery_location,brewery_name,id
0,"Minneapolis, MN",NorthGate Brewing,0
1,"Louisville, KY",Against the Grain Brewery,1
2,"Framingham, MA",Jack's Abby Craft Lagers,2
3,"San Diego, CA",Mike Hess Brewing Company,3
4,"San Francisco, CA",Fort Point Beer Company,4


In [70]:
breweries['city']=breweries['brewery_location'].apply(lambda x:x.split(',')[0])
breweries['state']=breweries['brewery_location'].apply(lambda x:x.split(',')[1])
breweries.head()

,brewery_location,brewery_name,id,city,state
0,"Minneapolis, MN",NorthGate Brewing,0,Minneapolis,MN
1,"Louisville, KY",Against the Grain Brewery,1,Louisville,KY
2,"Framingham, MA",Jack's Abby Craft Lagers,2,Framingham,MA
3,"San Diego, CA",Mike Hess Brewing Company,3,San Diego,CA
4,"San Francisco, CA",Fort Point Beer Company,4,San Francisco,CA


In [74]:
breweries = breweries[["brewery_name","city","state"]]
breweries.rename(inplace=True,columns={'brewery_name':'name'})
breweries.head()

,name,city,state
0,NorthGate Brewing,Minneapolis,MN
1,Against the Grain Brewery,Louisville,KY
2,Jack's Abby Craft Lagers,Framingham,MA
3,Mike Hess Brewing Company,San Diego,CA
4,Fort Point Beer Company,San Francisco,CA
